# Content Based Recommendation

1. Membuat Metada Per-Item

In [1]:
import joblib

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import re

from sklearn.feature_extraction.text import CountVectorizer

import pymongo

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics

from wordcloud import WordCloud,STOPWORDS 

# SETTING
plt.style.use('ggplot')
#display maximum column and rows
pd.options.display.max_columns = 999
pd.options.display.min_rows = 999
pd.options.display.max_colwidth = 999
stopword = STOPWORDS
stopword.add('im')
stopword.add('youre')


red = '#ef4b4b'
black = '#584b42'
green = '#a0cc78'
yellow_ = '#f6d365'
blue_ = '#49beb7'

#INIT


#METHODS
def sep():
    print(100*'-')

def connectMongoDB(dbname):
    client = pymongo.MongoClient('localhost', 27017)
    db = client[dbname]
    return db

dbname = 'clothing_fit'
db = connectMongoDB(dbname)

In [4]:
rent = list(db['rentCollection'].find())
df_rent = pd.DataFrame(rent)
df_rent

# MENGECEK NULL
print("banyak data : ", len(df_rent))
print("columns : ", df_rent.columns)

df_rent.info()


banyak data :  20027
columns :  Index(['_id', 'fit', 'user_id', 'bust size', 'item_id', 'weight', 'rating',
       'rented for', 'review_text', 'body type', 'review_summary', 'category',
       'height', 'size', 'age', 'review_date', 'sentiment_', 'height_cm',
       'weight_kg', 'age_', 'review_date_', 'sentiment_summary',
       'sentiment_text'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20027 entries, 0 to 20026
Data columns (total 23 columns):
_id                  20027 non-null object
fit                  20027 non-null object
user_id              20027 non-null object
bust size            18126 non-null object
item_id              20027 non-null object
weight               16866 non-null object
rating               20024 non-null float64
rented for           20027 non-null object
review_text          20027 non-null object
body type            18517 non-null object
review_summary       20027 non-null object
category             20027 non-null object


In [5]:
# SELEKSI KOLOM yang Mungkin Akan diproses

cols_needed = ['_id', 'fit', 'user_id', 'bust size', 'item_id', 'rating',
       'rented for', 'body type', 'category', 'size', 'sentiment_', 'height_cm',
       'weight_kg', 'age_', 'review_date_', 'sentiment_summary',
       'sentiment_text']
df_rent = df_rent[cols_needed].copy()
df_rent.head()
df_rent.isnull().sum()

_id                     0
fit                     0
user_id                 0
bust size            1901
item_id                 0
rating                  3
rented for              0
body type            1510
category                0
size                    0
sentiment_              0
height_cm               0
weight_kg            3161
age_                    0
review_date_            0
sentiment_summary       0
sentiment_text          0
dtype: int64

In [6]:
df_rent = df_rent.dropna(subset=['body type', 'weight_kg'])
len(df_rent)
df_rent['rented for'].unique()

array(['vacation', 'other', 'formal affair', 'wedding', 'date',
       'everyday', 'party', 'work'], dtype=object)

In [7]:
df_rent['id'] = range(15982)
df_rent.isnull().sum()
df_rent = df_rent.set_index('id')
df_rent.head()

_id  fit user_id bust size  item_id  rating  \
id                                                                     
0   5e2ab8ce9f84980caca61bf7  fit  420272       34d  2260466    10.0   
1   5e2ab8ce9f84980caca61bf8  fit  273551       34b   153475    10.0   
2   5e2ab8ce9f84980caca61bfa  fit  909926       34c   126335     8.0   
3   5e2ab8ce9f84980caca61bfb  fit  151944       34b   616682    10.0   
4   5e2ab8ce9f84980caca61bfc  fit  734848       32b   364092     8.0   

       rented for          body type category  size sentiment_  height_cm  \
id                                                                          
0        vacation          hourglass   romper    14   positive      173.0   
1           other  straight & narrow     gown    12   positive      168.0   
2   formal affair               pear    dress     8   positive      165.0   
3         wedding           athletic     gown    12   positive      175.0   
4            date           athletic    dress     8   positive      173.0   

    weight_kg  age_ review_date_ sentiment_summary sentiment_text  
id                                                                 
0        62.0  28.0   2016-04-20          positive       positive  
1        60.0  36.0   2013-06-18          positive       positive  
2        61.0  34.0   2014-02-12          positive       positive  
3        66.0  27.0   2016-09-26          positive       positive  
4        63.0  45.0   2016-04-30           neutral       positive

In [8]:
# TRANSFORMING one hot untuk rented for
encRentfor = preprocessing.OneHotEncoder()

print('banyak data : ', df_rent.shape)
encRentfor.fit(df_rent[['rented for']])
x = encRentfor.transform(df_rent[['rented for']]).toarray()
print(encRentfor.categories_)
print(x)
cols = encRentfor.categories_[0]
df_rentfor = pd.DataFrame(x, columns = cols)
print(df_rentfor.shape)
print(df_rent.shape)

df_rentfor.isnull().sum()

banyak data :  (15982, 17)
[array(['date', 'everyday', 'formal affair', 'other', 'party', 'vacation',
       'wedding', 'work'], dtype=object)]
[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(15982, 8)
(15982, 17)


date             0
everyday         0
formal affair    0
other            0
party            0
vacation         0
wedding          0
work             0
dtype: int64

In [10]:
encRentfor.transform([['other']]).toarray()[0].tolist()

[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]

In [11]:
joblib.dump(encRentfor, 'onehot_rentfor')

['onehot_rentfor']

In [12]:
# RUN SEKALI SAJA!
print("banyak data : ", df_rentfor.shape)
print(df_rentfor.shape)
print(df_rent.shape)
df_rent = pd.concat([df_rent, df_rentfor], axis=1) ###########!!! concat 1 kali
df_rent.head()
print(df_rentfor.shape)
print(df_rent.shape)

banyak data :  (15982, 8)
(15982, 8)
(15982, 17)
(15982, 8)
(15982, 25)


In [13]:
df_rent.isnull().sum()

_id                    0
fit                    0
user_id                0
bust size            679
item_id                0
rating                 0
rented for             0
body type              0
category               0
size                   0
sentiment_             0
height_cm              0
weight_kg              0
age_                   0
review_date_           0
sentiment_summary      0
sentiment_text         0
date                   0
everyday               0
formal affair          0
other                  0
party                  0
vacation               0
wedding                0
work                   0
dtype: int64

In [14]:
print(len(df_rent))
print(df_rent.columns)
df_rent.isnull().sum()
df_rentfor.isnull().sum()

15982
Index(['_id', 'fit', 'user_id', 'bust size', 'item_id', 'rating', 'rented for',
       'body type', 'category', 'size', 'sentiment_', 'height_cm', 'weight_kg',
       'age_', 'review_date_', 'sentiment_summary', 'sentiment_text', 'date',
       'everyday', 'formal affair', 'other', 'party', 'vacation', 'wedding',
       'work'],
      dtype='object')


date             0
everyday         0
formal affair    0
other            0
party            0
vacation         0
wedding          0
work             0
dtype: int64

In [15]:
xx = df_rent.groupby(['fit','item_id','size']).mean()

# melihat kondisi masing2 item
df_fit = xx.loc[('fit')]
df_small = xx.loc[('small')]
df_large = xx.loc[('large')]

print("fit : ", len(df_fit))
print("small : ", len(df_small))
print("large : ", len(df_large))
# plt.figure(figsize=(15,15))
# sb.heatmap(xx.corr(), annot=True)

fit :  7343
small :  1807
large :  1703


In [16]:
# TRANSFORMING one hot untuk fit

encLabelfit = preprocessing.OneHotEncoder()
encLabelfit.fit(df_rent[['fit']])
fit_array = encLabelfit.transform(df_rent[['fit']]).toarray()
print(encLabelfit.categories_[0])
fit_array

df_fit = pd.DataFrame(fit_array, columns = encLabelfit.categories_[0] )
df_fit.head()

['fit' 'large' 'small']


fit  large  small
0  1.0    0.0    0.0
1  1.0    0.0    0.0
2  1.0    0.0    0.0
3  1.0    0.0    0.0
4  1.0    0.0    0.0

In [ ]:
joblib.dump(encLabelfit, 'onehot_fit')

In [17]:
df_fit = pd.DataFrame(df_fit, index = df_rent.index)
df_fit.index
df_rent.index

counter=0
for i in df_fit.index:
    if i != df_rent.index[counter]:
        print("beda")
    counter += 1

In [18]:
print((df_fit.shape))
print((df_rent.shape))

df_rent = pd.concat([df_rent, df_fit], axis=1)
df_rent.head()

print((df_fit.shape))
print((df_rent.shape))

(15982, 3)
(15982, 25)
(15982, 3)
(15982, 28)


In [19]:
df_rent.isnull().sum()

_id                    0
fit                    0
user_id                0
bust size            679
item_id                0
rating                 0
rented for             0
body type              0
category               0
size                   0
sentiment_             0
height_cm              0
weight_kg              0
age_                   0
review_date_           0
sentiment_summary      0
sentiment_text         0
date                   0
everyday               0
formal affair          0
other                  0
party                  0
vacation               0
wedding                0
work                   0
fit                    0
large                  0
small                  0
dtype: int64

# transform metadata matrix dengan index (item_id x size)
karena masing2 baju dengan ukuran berbeda menandakan kecocokan item yang berbeda

In [20]:
df_rent.isnull().sum()

_id                    0
fit                    0
user_id                0
bust size            679
item_id                0
rating                 0
rented for             0
body type              0
category               0
size                   0
sentiment_             0
height_cm              0
weight_kg              0
age_                   0
review_date_           0
sentiment_summary      0
sentiment_text         0
date                   0
everyday               0
formal affair          0
other                  0
party                  0
vacation               0
wedding                0
work                   0
fit                    0
large                  0
small                  0
dtype: int64

In [21]:
df_metadata = df_rent.groupby(['item_id','size']).mean()
df_metadata['item_unique'] = df_metadata.index
print(df_metadata.shape)
df_metadata.head()

(9377, 16)


rating  height_cm  weight_kg  age_  date  everyday  \
item_id size                                                       
1000254 4       10.0      157.0       52.0  26.0   0.0       0.0   
        12      10.0      163.0       59.0  45.0   0.0       0.0   
        16      10.0      165.0       64.0  27.0   0.0       0.0   
1000270 1       10.0      157.0       49.0  41.0   0.0       0.0   
        4       10.0      165.0       56.0  30.5   0.0       0.0   

              formal affair  other  party  vacation  wedding  work  fit  \
item_id size                                                              
1000254 4               0.0    0.0    0.0       0.0      1.0   0.0  1.0   
        12              0.0    0.0    1.0       0.0      0.0   0.0  0.0   
        16              0.0    0.0    1.0       0.0      0.0   0.0  0.0   
1000270 1               0.0    1.0    0.0       0.0      0.0   0.0  1.0   
        4               0.0    0.0    1.0       0.0      0.0   0.0  0.5   

              large  small    item_unique  
item_id size                               
1000254 4       0.0    0.0   (1000254, 4)  
        12      0.0    1.0  (1000254, 12)  
        16      0.0    1.0  (1000254, 16)  
1000270 1       0.0    0.0   (1000270, 1)  
        4       0.0    0.5   (1000270, 4)

In [22]:
df_metadata['vote_count'] = df_rent.groupby(['item_id', 'size']).count()['rating']
df_metadata.head()

rating  height_cm  weight_kg  age_  date  everyday  \
item_id size                                                       
1000254 4       10.0      157.0       52.0  26.0   0.0       0.0   
        12      10.0      163.0       59.0  45.0   0.0       0.0   
        16      10.0      165.0       64.0  27.0   0.0       0.0   
1000270 1       10.0      157.0       49.0  41.0   0.0       0.0   
        4       10.0      165.0       56.0  30.5   0.0       0.0   

              formal affair  other  party  vacation  wedding  work  fit  \
item_id size                                                              
1000254 4               0.0    0.0    0.0       0.0      1.0   0.0  1.0   
        12              0.0    0.0    1.0       0.0      0.0   0.0  0.0   
        16              0.0    0.0    1.0       0.0      0.0   0.0  0.0   
1000270 1               0.0    1.0    0.0       0.0      0.0   0.0  1.0   
        4               0.0    0.0    1.0       0.0      0.0   0.0  0.5   

              large  small    item_unique  vote_count  
item_id size                                           
1000254 4       0.0    0.0   (1000254, 4)           1  
        12      0.0    1.0  (1000254, 12)           1  
        16      0.0    1.0  (1000254, 16)           1  
1000270 1       0.0    0.0   (1000270, 1)           1  
        4       0.0    0.5   (1000270, 4)           2

In [23]:
# Calculate C (mean vote dari semua data yang tersedia)

C = df_metadata['rating'].mean()
print(C)
# -------------------------------------------------------------------------
# Calculate the minimum number of votes required to be in the chart, m 
## persentil ke 90 dari angka vote keseluruhan data

m = df_metadata['vote_count'].quantile(0.90)
print(m)

# -------------------------------------------------------------------------
# Function that computes the weighted rating of each clothes

def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['rating']
    
    # Calculation based on the IMDB formula
    # jumlah Nvote * rating
    return (v/(v+m) * R) + (m/(m+v) * C)



9.072789099622456
3.0


In [24]:
# -------------------------------------------------------------------------
# Define a new feature 'score' and calculate its value with `weighted_rating()`

df_metadata['score'] = df_metadata.apply(weighted_rating, axis=1)
df_metadata.head()
# q_clothes.shape
# q_clothes

rating  height_cm  weight_kg  age_  date  everyday  \
item_id size                                                       
1000254 4       10.0      157.0       52.0  26.0   0.0       0.0   
        12      10.0      163.0       59.0  45.0   0.0       0.0   
        16      10.0      165.0       64.0  27.0   0.0       0.0   
1000270 1       10.0      157.0       49.0  41.0   0.0       0.0   
        4       10.0      165.0       56.0  30.5   0.0       0.0   

              formal affair  other  party  vacation  wedding  work  fit  \
item_id size                                                              
1000254 4               0.0    0.0    0.0       0.0      1.0   0.0  1.0   
        12              0.0    0.0    1.0       0.0      0.0   0.0  0.0   
        16              0.0    0.0    1.0       0.0      0.0   0.0  0.0   
1000270 1               0.0    1.0    0.0       0.0      0.0   0.0  1.0   
        4               0.0    0.0    1.0       0.0      0.0   0.0  0.5   

              large  small    item_unique  vote_count     score  
item_id size                                                     
1000254 4       0.0    0.0   (1000254, 4)           1  9.304592  
        12      0.0    1.0  (1000254, 12)           1  9.304592  
        16      0.0    1.0  (1000254, 16)           1  9.304592  
1000270 1       0.0    0.0   (1000270, 1)           1  9.304592  
        4       0.0    0.5   (1000270, 4)           2  9.443673

In [25]:
df_metadata['item_id_size'] = df_metadata['item_unique'].apply(lambda x : x[0] +","+str(int(x[1])))
# df_metadata['item_unique'] = df_metadata['item_id_size']
df_metadata.head()
print(len(df_metadata))
df_metadata = df_metadata.set_index('item_unique', inplace=False)
df_metadata.head()

9377


rating  height_cm  weight_kg  age_  date  everyday  \
item_unique                                                         
(1000254, 4)     10.0      157.0       52.0  26.0   0.0       0.0   
(1000254, 12)    10.0      163.0       59.0  45.0   0.0       0.0   
(1000254, 16)    10.0      165.0       64.0  27.0   0.0       0.0   
(1000270, 1)     10.0      157.0       49.0  41.0   0.0       0.0   
(1000270, 4)     10.0      165.0       56.0  30.5   0.0       0.0   

               formal affair  other  party  vacation  wedding  work  fit  \
item_unique                                                                
(1000254, 4)             0.0    0.0    0.0       0.0      1.0   0.0  1.0   
(1000254, 12)            0.0    0.0    1.0       0.0      0.0   0.0  0.0   
(1000254, 16)            0.0    0.0    1.0       0.0      0.0   0.0  0.0   
(1000270, 1)             0.0    1.0    0.0       0.0      0.0   0.0  1.0   
(1000270, 4)             0.0    0.0    1.0       0.0      0.0   0.0  0.5   

               large  small  vote_count     score item_id_size  
item_unique                                                     
(1000254, 4)     0.0    0.0           1  9.304592    1000254,4  
(1000254, 12)    0.0    1.0           1  9.304592   1000254,12  
(1000254, 16)    0.0    1.0           1  9.304592   1000254,16  
(1000270, 1)     0.0    0.0           1  9.304592    1000270,1  
(1000270, 4)     0.0    0.5           2  9.443673    1000270,4

In [26]:
# ada 9377 item unik (id x size)
df_metadata['counter'] = range(len(df_metadata))
df_metadata.tail(3)

rating  height_cm  weight_kg  age_  date  everyday  \
item_unique                                                        
(999837, 8)      9.0      162.5       55.0  34.0   0.0       0.0   
(999837, 12)     9.6      161.4       60.2  28.0   0.0       0.0   
(999837, 16)     9.0      168.0       60.5  31.5   0.0       0.0   

              formal affair  other  party  vacation  wedding  work  fit  \
item_unique                                                               
(999837, 8)             0.0    1.0    0.0       0.0      0.0   0.0  0.0   
(999837, 12)            0.0    0.0    0.0       0.0      1.0   0.0  0.4   
(999837, 16)            0.0    0.0    0.5       0.0      0.5   0.0  0.5   

              large  small  vote_count     score item_id_size  counter  
item_unique                                                             
(999837, 8)     0.0    1.0           2  9.043673     999837,8     9374  
(999837, 12)    0.0    0.6           5  9.402296    999837,12     9375  
(999837, 16)    0.0    0.5           2  9.043673    999837,16     9376

In [27]:
df_metadata.to_pickle('items_metadata')

# Membentuk DF yang hanya berisi fitur yang akan diuji dalam rekomendasi

In [28]:
cols = ['score', 'height_cm', 'weight_kg', 'age_', 'date', 'everyday','formal affair', 'other', 'party', 'vacation', 'wedding', 'work', 'fit','large', 'small']
# dari kolom ini, input user adalah
df = df_metadata[cols].copy()


In [29]:
df.isnull().sum()


score            0
height_cm        0
weight_kg        0
age_             0
date             0
everyday         0
formal affair    0
other            0
party            0
vacation         0
wedding          0
work             0
fit              0
large            0
small            0
dtype: int64

# Menghitung Similarity

In [30]:
# Import cosine
from sklearn.metrics.pairwise import cosine_similarity

# Compute the cosine similarity matrix
cosine_sim = cosine_similarity(df, df)

df_cosine_sim = pd.DataFrame(cosine_sim)
df_cosine_sim

0         1         2         3         4         5         6     \
0     1.000000  0.994778  0.998691  0.995931  0.999779  0.995502  0.998939   
1     0.994778  1.000000  0.994510  0.998945  0.996554  0.999872  0.996057   
2     0.998691  0.994510  1.000000  0.993452  0.998862  0.994603  0.999851   
3     0.995931  0.998945  0.993452  1.000000  0.997153  0.999412  0.995145   
4     0.999779  0.996554  0.998862  0.997153  1.000000  0.997025  0.999302   
5     0.995502  0.999872  0.994603  0.999412  0.997025  1.000000  0.996187   
6     0.998939  0.996057  0.999851  0.995145  0.999302  0.996187  1.000000   
7     0.999185  0.997105  0.999408  0.996658  0.999669  0.997323  0.999802   
8     0.995149  0.990264  0.998698  0.987479  0.995236  0.990077  0.998047   
9     0.999679  0.994583  0.997246  0.996655  0.999451  0.995481  0.997733   
10    0.999631  0.995465  0.997408  0.997239  0.999599  0.996223  0.997961   
11    0.999674  0.994572  0.999654  0.994689  0.999575  0.995003  0.999647   
12    0.998899  0.996455  0.999717  0.995563  0.999315  0.996615  0.999963   
13    0.998304  0.995860  0.999766  0.994421  0.998727  0.995926  0.999884   
14    0.999899  0.995576  0.998432  0.996770  0.999866  0.996257  0.998844   
15    0.999940  0.994591  0.999105  0.995386  0.999731  0.995234  0.999241   
16    0.999243  0.995945  0.999774  0.995366  0.999464  0.996227  0.999948   
17    0.999021  0.996294  0.999782  0.995460  0.999378  0.996461  0.999976   
18    0.998674  0.995113  0.999931  0.993988  0.998913  0.995232  0.999896   
19    0.991014  0.987812  0.996402  0.983358  0.991519  0.987174  0.995641   
20    0.990518  0.999276  0.990807  0.997338  0.992913  0.998925  0.992807   
21    0.998937  0.997571  0.999186  0.996991  0.999535  0.997756  0.999704   
22    0.998646  0.998465  0.997203  0.999082  0.999363  0.998892  0.998209   
23    0.997694  0.999383  0.997055  0.999103  0.998803  0.999562  0.998180   
24    0.999393  0.997441  0.997798  0.998389  0.999755  0.998040  0.998584   
25    0.996084  0.999704  0.996352  0.998471  0.997556  0.999635  0.997601   
26    0.999819  0.996081  0.999127  0.996607  0.999928  0.996587  0.999475   
27    0.998961  0.997121  0.999512  0.996353  0.999474  0.997291  0.999892   
28    0.999531  0.997057  0.999148  0.997131  0.999908  0.997415  0.999609   
29    0.999165  0.996132  0.999779  0.995467  0.999467  0.996362  0.999969   
...        ...       ...       ...       ...       ...       ...       ...   
9347  0.997667  0.999373  0.996898  0.999167  0.998724  0.999623  0.998064   
9348  0.998182  0.999052  0.997290  0.999087  0.999106  0.999345  0.998358   
9349  0.996895  0.999393  0.995301  0.999773  0.998074  0.999726  0.996746   
9350  0.987409  0.998000  0.986019  0.997168  0.990046  0.997858  0.988615   
9351  0.999599  0.993524  0.999528  0.993796  0.999308  0.994018  0.999379   
9352  0.998950  0.995010  0.999974  0.994148  0.999152  0.995145  0.999917   
9353  0.997254  0.999175  0.997766  0.997941  0.998464  0.999112  0.998688   
9354  0.991119  0.990962  0.996341  0.986263  0.992163  0.990190  0.996063   
9355  0.980144  0.989757  0.986769  0.982565  0.982978  0.988219  0.987656   
9356  0.997261  0.996045  0.999376  0.993915  0.997937  0.995928  0.999549   
9357  0.998227  0.994330  0.999893  0.992952  0.998423  0.994412  0.999758   
9358  0.998940  0.996890  0.999563  0.996087  0.999425  0.997056  0.999890   
9359  0.999873  0.995902  0.998491  0.996981  0.999908  0.996548  0.998938   
9360  0.998724  0.994448  0.999938  0.993485  0.998831  0.994634  0.999835   
9361  0.998488  0.998460  0.996751  0.999293  0.999214  0.998946  0.997844   
9362  0.996783  0.991312  0.999419  0.989369  0.996775  0.991261  0.998833   
9363  0.999696  0.996489  0.999177  0.996772  0.999917  0.996924  0.999563   
9364  0.999297  0.997259  0.997391  0.998497  0.999676  0.997872  0.998212   
9365  0.996899  0.997363  0.993380  0.999528  0.997587  0.998183  0.994897   
9366  0.998861  0.993422  0.9998

In [31]:
# Compute the pearson matrix

pearson_sim = (df.T.corr())

In [32]:
pearson_sim = np.array(pearson_sim)

In [33]:
print(df.shape)
print(cosine_sim.shape)
print(pearson_sim.shape)
# print(pearson_sim[0])

(9377, 15)
(9377, 9377)
(9377, 9377)


In [34]:
id = 999837
size = 12
id_size = str(id)+','+str(size)
selected = df_metadata[df_metadata['item_id_size'] == id_size]

# Get the index of the clothes that matches the id and size
index = selected['counter']
print(index)
df_metadata.iloc[index]

# Get the pairwsie similarity scores of all clothes with that clothes
sim_scores = list(enumerate(cosine_sim[index][0]))
pear_scores = list(enumerate(pearson_sim[index][0]))


# # Sort the clothes based on the similarity scores & pearson
sim_scores_sort = sorted(sim_scores, key=lambda x: x[1], reverse=True)
pear_scores_sort = sorted(pear_scores, key=lambda x: x[1], reverse=True)

# # print(sim_scores_sort)
# # Get the scores of the 10 most similar clothes
sim_scores_top = sim_scores_sort[1:11]
# print(sim_scores_top)
sep()
pear_scores_top = pear_scores_sort[1:11]
# print(pear_scores_top)

#  # Get the clothes indices
clothes_indices = [i[0] for i in sim_scores_top]
clothes_indices2 = [i[0] for i in pear_scores_top]
print(clothes_indices)
print(clothes_indices2)

item_unique
(999837, 12)    9375
Name: counter, dtype: int32
----------------------------------------------------------------------------------------------------
[2405, 7250, 2333, 2767, 1430, 1699, 3109, 16, 961, 9031]
[2405, 7250, 2333, 1430, 2767, 1699, 3109, 16, 4865, 961]


In [35]:
counter=0
for i in clothes_indices:
    counter+=1
    print(counter)
    print(df_metadata[df_metadata['counter'] == i]['score'])

1
item_unique
(1478427, 12)    9.443673
Name: score, dtype: float64
2
item_unique
(397303, 12)    8.804592
Name: score, dtype: float64
3
item_unique
(1463604, 20)    9.304592
Name: score, dtype: float64
4
item_unique
(154002, 17)    9.443673
Name: score, dtype: float64
5
item_unique
(1314701, 14)    9.304592
Name: score, dtype: float64
6
item_unique
(1352656, 8)    9.304592
Name: score, dtype: float64
7
item_unique
(1618192, 8)    9.304592
Name: score, dtype: float64
8
item_unique
(1001122, 12)    9.304592
Name: score, dtype: float64
9
item_unique
(1238932, 8)    9.402296
Name: score, dtype: float64
10
item_unique
(909677, 16)    8.804592
Name: score, dtype: float64


In [36]:
df.shape

(9377, 15)

In [37]:
df.columns

Index(['score', 'height_cm', 'weight_kg', 'age_', 'date', 'everyday',
       'formal affair', 'other', 'party', 'vacation', 'wedding', 'work', 'fit',
       'large', 'small'],
      dtype='object')

In [38]:
df_embed = df.copy()
## dari 14 jadi 3
## 15 -> ['score', 'height_cm', 'weight_kg', 'age_', 'date', 'everyday','formal affair',
#        'other', 'party', 'vacation', 'wedding', 'work', 'fit', 'large',
#        'small']
## 3 -> [ embed1, embed2, embed3 ] 
# explained_variance_ratio_ -> (0.994672628366279)

from sklearn.decomposition import PCA
pca = PCA(n_components=3)
pca.fit(df_embed)
df_embed = pca.transform(df_embed)
print(pca.explained_variance_ratio_.sum())
df_embed

0.9941869468407114


array([[-13.29453452,  -6.81039504,   4.9731126 ],
       [ -3.01218534,  10.98207183,   0.08118066],
       [  0.24808184,  -7.52016869,  -0.03654709],
       ...,
       [ -8.23650163,   0.49660662,   0.03613582],
       [ -4.1349154 ,  -5.9273446 ,   2.53851704],
       [ -1.94452288,  -2.88572318,  -3.93278307]])

In [39]:
# Eksport PCA decomposer
import joblib


# MENYIMPAN DENGAN JOBLIB
joblib.dump(pca, 'PCA_15_to_3')
joblib.dump(df_embed, 'items_embed')

['items_embed']

In [40]:
# membuat matrix similarity

df_embed.shape
cos_score_one = cosine_similarity(df_embed,[[-13.29475289,  -6.8098584 ,   4.97293176]])
cos_score_one.shape

cosine_sim_matrix = cosine_similarity(df_embed, df_embed)

df_cosine_sim_matrix = pd.DataFrame(cosine_sim_matrix)

In [ ]:
df_cosine_sim_matrix.to_pickle('content_similarity')

In [46]:
# ['score', 'height_cm', 'weight_kg', 'age_', 'date', 'formal affair',
#        'other', 'party', 'vacation', 'wedding', 'work', 'fit', 'large',
#        'small']
skor = 10
tinggi = 155
berat = 65
umur = 22
date = 0
everyday = 0
formal = 0
other = 0
party = 0
vacation = 0
wedding = 1
work = 0
fit = 1
large = 0
small = 0

my_input = [skor, tinggi, berat, umur, date, everyday,
         formal, other, party, vacation, wedding, 
         work, fit, large, small]



In [47]:
# LOAD PCA

my_pca = joblib.load('PCA_15_to_3')
input_embedding = my_pca.transform([my_input])

In [48]:
#
cos_score_one = cosine_similarity(df_embed,input_embedding)
cos_score_one

array([[ 0.61891305],
       [-0.70137701],
       [ 0.75349124],
       ...,
       [ 0.06040713],
       [ 0.85679592],
       [-0.02390017]])

In [49]:
id_size = ('999837', 12)
# id_size = None
n = 5

if (id_size) :
    id_size = str(id_size[0])+','+str(id_size[1])
    selected = df_metadata[df_metadata['item_id_size'] == id_size]

    # Get the index of the clothes that matches the id and size
    index = selected['counter']
    print("your previous favs : ",index[0])
    df_metadata.iloc[index]

    # Get the pairwsie similarity scores of all clothes with that clothes
    sim_scores = list(enumerate(cosine_sim_matrix[index][0]))
else : 
    input_embedding = my_pca.transform([my_input])
    sim_scores = enumerate(cosine_similarity(df_embed, input_embedding))

# # Sort the clothes based on the similarity scores & pearson
sim_scores_sort = sorted(sim_scores, key=lambda x: x[1], reverse=True)
# pear_scores_sort = sorted(pear_scores, key=lambda x: x[1], reverse=True)

# # print(sim_scores_sort)
# # Get the scores of the 10 most similar clothes
sim_scores_top = sim_scores_sort[1:n+1]
# print(sim_scores_top)
sep()

# pear_scores_top = pear_scores_sort[1:11]
# print(pear_scores_top)

#  # Get the clothes indices
clothes_indices = [i[0] for i in sim_scores_top]
# clothes_indices2 = [i[0] for i in pear_scores_top]
print("Based on Your Previous : ")
print(clothes_indices)
# print(clothes_indices2)

your previous favs :  9375
----------------------------------------------------------------------------------------------------
Based on Your Previous : 
[8374, 1670, 8589, 7787, 277]


In [50]:
def getItemInCounter(indices):
    result = pd.DataFrame()
    for i in indices:
        tem = df_metadata[df_metadata['counter'] == i]
        result = pd.concat([result,tem])
    return result

In [65]:
clothes_indices

[8374, 1670, 8589, 7787, 277]

In [61]:
hasil_rekom = getItemInCounter(clothes_indices)
hasil_rekom = hasil_rekom.values

In [64]:

hasil_rekom[0][2]

59.0

# Evaluation

In [ ]:
import recmetrics

pred = [
        [5072, 5736, 4636, 8446, 2966, 4186, 4816, 7927, 1459, 7796],
        [8374, 1670, 8589, 7787, 277, 3812, 1634, 668, 70, 2364]
]

df_embed_ = pd.DataFrame(df_embed)
feature_df = df_embed_.iloc[:,:]

print(df_metadata.shape)
print(df_embed.shape)
print(feature_df.shape)

In [ ]:
recmetrics.personalization(predicted=pred)

# intra-list similarity

If a recommender system is recommending lists of very similar items to single users (for example, a user receives only recommendations of wedding clothes), then the intra-list similarity will be high.

In [ ]:
recmetrics.intra_list_similarity(pred, feature_df)

In [ ]:
# def getRecommendation(id_size=None, my_input=None):
    

# Memfilter item yang positive

In [ ]:
# memfilter item yang memiliki sentimen positif terhadap items
df_rent_pos = df_rent[df_rent.sentiment_text == 'positive'][df_rent.sentiment_summary == 'positive'][df_rent.fit == 'fit']
df_rent_pos2 = df_rent[df_rent.sentiment_text == 'positive']
df_rent_pos3 = df_rent[df_rent.sentiment_summary == 'positive']

# df_rent_neg = df_rent[df_rent.sentiment_text == 'negative']


In [ ]:
df_rent_pos.groupby(['rating']).count()
df_rent_pos[df_rent_pos['rating']==2]

In [ ]:
df_rent_pos2.groupby(['rating']).count()
# df_rent_pos2[df_rent_pos['rating']==2]

In [ ]:
df_rent_pos3.groupby(['rating']).count()
# df_rent_pos2[df_rent_pos['rating']==2]

In [ ]:
print(df_metadata.columns)
df_metadata.head()


In [ ]:
# Dump df_metadata
occasion = ('date','everyday','formal', 'other', 'party', 
            'vacation', 'wedding', 'work')
kat = (df_metadata.iloc[:,4:12])
kat = kat.values

fititem = ('fit', 'large', 'small')
fitcol = (df_metadata.iloc[:,12:15])
fitcol = fitcol.values

occlist = []
for row in kat:
    occ = []
    for j in range(len(row)):
        if row[j] != 0:
            occ.append(occasion[j])
    occlist.append(occ)
print(len(occlist))
df_metadata['occasion'] = occlist

fitlist = []
for row in fitcol:
    ft = []
    for j in range(len(row)):
        if row[j] != 0:
            ft.append(fititem[j])
    fitlist.append(ft)
print(len(fitlist))
df_metadata['fit_feedback'] = fitlist


df_metadata.to_pickle('items_metadata')

In [ ]:
occasion = ('date','formal', 'other', 'party', 
            'vacation', 'wedding', 'work')
rentfor = np.zeros(7)
print(rentfor)
print(occasion.index('wedding'))
ceklis = occasion.index('wedding')
rentfor[ceklis] = 1
print(rentfor)


awal = [23,23]
dua= [134]
awal.extend(dua)
awal.extend(rentfor)
awal

In [ ]:
df_rekom_item = getItemInCounter(clothes_indices)
rekom_items = df_rekom_item.to_dict('records')
rekom_items